# Language Model

In [1]:
## Update this later with tuned parameters

#get embedding model and compute loss (for use in hyperparameter tuning)
import w2v_for_lm
import time

for i in range(1,2):
    print(i)
    start = time.time()
    epochs = i
    vec_size = 10
    window = 5
    model = w2v_for_lm.retrieve_model_na(epochs, vec_size, window)
    end = time.time()
    print(end - start)
    loss = model.get_latest_training_loss()
    # perplexity = 2**loss
    print(loss)

/home/ejhaselden/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


1
33.6106173992157
4437299.5


In [2]:
test = w2v_for_lm.retrieve_query()
wordlist = test['cn'].tolist()


In [3]:
# Standard python helper libraries.
from __future__ import print_function
from __future__ import division
import os, sys, time
import collections
import itertools

# Numerical manipulation libraries.
import numpy as np
from scipy import stats, optimize

# NLTK is the Natural Language Toolkit, and contains several language datasets
# as well as implementations of many popular NLP algorithms.
# HINT: You should look at what is available here when thinking about your project!
import nltk

# Helper libraries (see the corresponding py files in this notebook's directory).
import utils, vocabulary

# import segment

utils.require_package("tqdm")  # for nice progress bars
from tqdm import tqdm as ProgressBar

# # Bokeh for plotting.
utils.require_package("bokeh")
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

Loading BokehJS ...

In [4]:
##NOT FROM HERE
import re
# Word processing functions
def canonicalize_digits(word):
    if any([c.isalpha() for c in word]): return word
    word = re.sub("\d", "DG", word)
    if word.startswith("DG"):
        word = word.replace(",", "") # remove thousands separator
    return word

def canonicalize_word(word, wordset=None, digits=True):
    word = word.lower()
    if digits:
        if (wordset != None) and (word in wordset): return word
        word = canonicalize_digits(word) # try to canonicalize numbers
    if (wordset == None) or (word in wordset):
        return word
    else:
        return constants.UNK_TOKEN

def canonicalize_words(words, **kw):
    return [canonicalize_word(word, **kw) for word in words]

In [5]:
# corpus = nltk.corpus.brown
# corpus = embedded_corpus

# "canonicalize_word" performs a few tweaks to the token stream of
# the corpus.  For example, it replaces digits with DG allowing numbers
# to aggregate together when we count them below.
# You can read the details in utils.py if you're really curious.


# token_feed = (utils.canonicalize_word(w) for w in corpus.words())
token_feed = (canonicalize_word(w) for w in wordlist)



# # Collect counts of tokens and assign wordids.
vocab = vocabulary.Vocabulary(token_feed, progressbar=ProgressBar)
# vocab = vocabulary.Vocabulary(token_feed)
print("Vocabulary size: {:,}".format(vocab.size))

# # Print out some (debugging) statistics to make sure everything went
# # as we expected.  (Unsurprisingly, you should see "the" as the most popular word.)
# print("Most common unigrams:")
# for word, count in vocab.unigram_counts.most_common(10):
#     print("\"{:s}\": {:,}".format(word, count))

779402it [00:15, 49455.99it/s]


Vocabulary size: 735,870


## N-Gram Language Models

The unigram model isn't a very good one - it doesn't model any previous context at all. On the other hand, we can't model _all_ of the preceding words, because that history could get prohibitively long and extremely sparse. As a compromise, we'll make a _Markov assumption_ and limit ourselves to a finite history of $n$ words.

For now, we'll build a trigram model, which considers the two preceding words:

$$ P(w_i\ |\ w_{i-1}, ..., w_0) \approx P(w_i\ |\ w_{i-1}, w_{i-2}) $$

**Exercise:** how many possible trigrams are there?

We'll need to store a table of the probabilities, indexed by triples $(w_i, w_{i-1}, w_{i-2})$. If we store every possible combination, it can get quite large. Assuming 8 bytes for each entry, we have:

In [6]:
# import psutil
# print("Vocab size:     {:10,} words".format(vocab.size))
# print("Unigrams need:  {:12.2f} kB".format(8 * (vocab.size ** 1) / (2**10)))
# print("Bigrams  need:  {:12.2f} MB".format(8 * (vocab.size ** 2) / (2**20)))
# print("Trigrams need:  {:12.2f} MB".format(8 * (vocab.size ** 3) / (2**20)))
# print("Available:      {:12.2f} MB".format(psutil.virtual_memory().available / (2**20)))

Well, that's no good. We can store all possible unigrams, but after that we're toast.

Thankfully, we don't have to store everything. Recall that most words only appear a handful of times, and that there are plenty of words in the English language that we don't see in our corpus at all. For bigrams and trigrams, the table will be quite sparse. We need only store the entries that we actually observe; the rest we can take to be zero, or estimate their values on the fly through smoothing or backoff.

**Observation:** When building language models, be sure to only keep non-zero counts in your datastructures and assume anything missing is 0.

**Exercise:** for a corpus of 1 million words and a vocabulary of 10000, what is the maximum number of bigrams that can be actually _observed_? How about trigrams?

### Constructing our Model

We'll represent our model with a nested map `context => word => probability`, where word is $w_i$ and for our trigram model, the context is the two preceding words $(w_{i-2}, w_{i-1})$.

First, we'll go through the corpus and compute raw trigram counts $c(abc)$, which we'll then normalize into probabilities:

$$  P_{abc} = P(w_i = c\ |\ w_{i-1} = b, w_{i-2} = a) = \frac{\mathrm{c(abc)}}{\sum_{c'}\mathrm{c(abc')}} = \frac{C_{abc}}{\sum_{c'} C_{abc'}} $$

Here's the code for our model:

In [7]:
from collections import defaultdict

def normalize_counter(c):
    """Given a dictionary of <item, counts>, return <item, fraction>."""
    total = sum(c.values())
    return {w:float(c[w])/total for w in c}

class SimpleTrigramLM(object):
    def __init__(self, words):
        """Build our simple trigram model."""
        # Raw trigram counts over the corpus. 
        # c(w | w_1 w_2) = self.counts[(w_2,w_1)][w]
        self.counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    
        # Iterate through the word stream once.
        w_1, w_2 = None, None
        for word in words:
            if w_1 is not None and w_2 is not None:
                # Increment trigram count.
                self.counts[(w_2,w_1)][word] += 1
            # Shift context along the stream of words.
            w_2 = w_1
            w_1 = word
            
        # Normalize so that for each context we have a valid probability
        # distribution (i.e. adds up to 1.0) of possible next tokens.
        self.probas = defaultdict(lambda: defaultdict(lambda: 0.0))
        for context, ctr in self.counts.items():
            self.probas[context] = normalize_counter(ctr)
            
    def next_word_proba(self, word, seq):
        """Compute p(word | seq)"""
        context = tuple(seq[-2:])  # last two words
        return self.probas[context].get(word, 0.0)
    
    def predict_next(self, seq):
        """Sample a word from the conditional distribution."""
        context = tuple(seq[-2:])  # last two words
        pc = self.probas[context]  # conditional distribution
        words, probs = zip(*pc.items())  # convert to list
        return np.random.choice(words, p=probs)
    
    def score_seq(self, seq, verbose=False):
        """Compute log probability (base 2) of the given sequence."""
        score = 0.0
        count = 0
        # Start at third word, since we need a full context.
        for i in range(2, len(seq)):
            if (seq[i] == "<s>" or seq[i] == "</s>"):
                continue  # Don't count special tokens in score.
            s = np.log2(self.next_word_proba(seq[i], seq[i-2:i]))
            score += s
            count += 1
            # DEBUG
            if verbose:
                print("log P({:s} | {:s}) = {.03f}".format(seq[i], " ".join(seq[i-2:i]), s))
        return score, count


### Training

Let's train our model. We'll do a proper train-test split this time, so we can evaluate on unseen data. This means that we also have to fix our vocabulary based on the *training* set - which means that some unseen words in the test set will get replaced by `<unk>`. The `Vocabulary` helper class will take care of this for us.

In [8]:
# len(test.values)
# sent_lite = test.values[0:10]

In [9]:
split=0.8
sentences = np.array(list(wordlist), dtype=object)
fmt = (len(sentences), sum(map(len, sentences)))
print("Loaded {:,} sentences ({:g} tokens)".format(*fmt))


rng = np.random.RandomState()
rng.shuffle(sentences)  # in-place
split_idx = int(split * len(sentences))
train_sents = sentences[:split_idx]
test_sents = sentences[split_idx:]

for l in range(len(train_sents)):
    train_sents[l] = train_sents[l].split(", ")
for l in range(len(test_sents)):
    test_sents[l] = test_sents[l].split(", ")
# train_sents = train_sents.split(",")
# test_sents = test_sents.split(",")

fmt = (len(train_sents), sum(map(len, train_sents)))
print("Training set: {:,} sentences ({:,} tokens)".format(*fmt))
fmt = (len(test_sents), sum(map(len, test_sents)))
print("Test set: {:,} sentences ({:,} tokens)".format(*fmt))

#     return train_sentences, test_sentences

Loaded 779,402 sentences (1.43624e+08 tokens)
Training set: 623,521 sentences (9,230,562 tokens)
Test set: 155,881 sentences (2,305,637 tokens)


In [10]:
train_sents


array([list(['775641564', 'Pink', 'One Piece Garment', 'Choreography', 'Fun', 'Purple', 'Celebrities', 'Entertainment', 'Performing Arts', 'Arts Culture and Entertainment', 'Costume Design', 'Dress', 'Music', 'Fashion', 'Magenta', 'Fashion Design', 'Awards Ceremony', 'Performance Art']),
       list(['Poster', 'Penguin', 'Font']),
       list(['Performance', 'Entertainment', 'BFfulltakes_FTP', 'Performing Arts', 'keep', '775561332', 'Event', 'Green', 'Stage']),
       ...,
       list(['Purple', 'Fashion', 'Musician', 'Music Artist', 'Performing Arts', 'Arts Culture and Entertainment', 'Kevin Mazur', 'Audio Equipment', 'Artist', '775657894', 'keep', 'Hat', 'Pop Music', 'Music', 'Concert', 'Microphone', 'Celebrities', 'Entertainment']),
       list(['Arts Culture and Entertainment', 'Eyewear', '775656581', 'Vision Care', 'Smile', 'BFfulltakes_FTP']),
       list(['Concert', 'Magenta', 'Music Venue', 'Awards Ceremony', 'BFfulltakes_FTP', 'Performing Arts', 'Musical Instrument', 'Arts Cul

In [11]:
vocab = vocabulary.Vocabulary(canonicalize_word(w) for w in ProgressBar(utils.flatten(train_sents)))
print("Train set vocabulary: %d words" % vocab.size)

100%|██████████| 9230562/9230562 [00:29<00:00, 314541.91it/s]


Train set vocabulary: 39686 words


#### A note on Preprocessing

We didn't do this for the unigram LM, but when modeling sentences it's helpful to add special beginning-of-sentence (`<s>`) and end-of-sentence (`</s>`) tokens. 

This lets the model estimate the probability of a word appearing at the beginning of a sentence, and lets it model the end of a sentence properly, since periods or other punctuation aren't always an accurate guide (e.g. `"Dr."` or `"Yahoo!"`).

Our padded sentences will look like this:
```
<s> <s> the cat sat in the hat . </s>
```
**Exercise:** why do we add two `<s>` tokens at the beginning? (*Hint: this is specific to our trigram model.*)

In [12]:
def sents_to_tokens(sents):
    """Returns an flattened list of the words in the sentences, with padding for a trigram model."""
    padded_sentences = (["<s>", "<s>"] + s + ["</s>"] for s in sents)
    # This will canonicalize words, and replace anything not in vocab with <unk>
    return np.array([utils.canonicalize_word(w, wordset=vocab.wordset) 
                     for w in ProgressBar(utils.flatten(padded_sentences))], dtype=object)

train_tokens = sents_to_tokens(train_sents)
test_tokens = sents_to_tokens(test_sents)

t0 = time.time()
print("Building trigram LM...",)
lm = SimpleTrigramLM(train_tokens)
print("done in %.02f s" % (time.time() - t0))

100%|██████████| 2773280/2773280 [00:02<00:00, 1062332.45it/s]


Building trigram LM...
done in 24.23 s


In [13]:
len(test_tokens)

2773280

## Generating suggestions

We can seed the suggestion list with a single tag, and let the model provide other tags it believes are related.

Next steps: loop this over a set of rare tags, and also limit suggestions to non-rare responses

In [14]:
max_length = 10
num_sentences = 5
suggestion_seed = ['<s>', 'music']

for _ in range(num_sentences):
    seq = suggestion_seed
    for i in range(max_length):
        seq.append(lm.predict_next(seq))
        # Stop at end-of-sentence
#         if seq[-1] == "</s>": break
    print(" ".join(seq))
    print("[{1:d} tokens; log P(seq): {0:.02f}]".format(*lm.score_seq(seq)))
    print("")

<s> music fashion arts culture and entertainment mike coppola DGDGDGDG-DGDG-DGDG water bffulltakes_ftp electric blue font </s> <s>
[8 tokens; log P(seq): -40.32]

<s> music fashion arts culture and entertainment mike coppola DGDGDGDG-DGDG-DGDG water bffulltakes_ftp electric blue font </s> <s> <s> DGDGDGDGDGDGDGDGDG dancer musical theatre ballet scene musical event fashion awards ceremony
[17 tokens; log P(seq): -73.22]

<s> music fashion arts culture and entertainment mike coppola DGDGDGDG-DGDG-DGDG water bffulltakes_ftp electric blue font </s> <s> <s> DGDGDGDGDGDGDGDGDG dancer musical theatre ballet scene musical event fashion awards ceremony performance art performing arts musical instrument </s> <s> <s> arts culture and entertainment DGDGDGDGDGDGDGDGDG black little black dress
[24 tokens; log P(seq): -105.82]

<s> music fashion arts culture and entertainment mike coppola DGDGDGDG-DGDG-DGDG water bffulltakes_ftp electric blue font </s> <s> <s> DGDGDGDGDGDGDGDGDG dancer musical theatr

## Scoring our model

We'll score our model by running the `score_seq` function, which computes 

$$ \text{CE}_{\text{total}}(y, \hat{y}) = \sum_{i=2}^N \log_2 \hat{p}(w_i\ |\ w_{i-1}, w_{i-2}) $$

This is the cross-entropy loss, which is equal to $-1$ times the log-likelihood of the data under our model. As usual, we'll exponentiate to get the perplexity score:

In [15]:
log_p_data, num_real_tokens = lm.score_seq(train_tokens)
print("Train perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

log_p_data, num_real_tokens = lm.score_seq(test_tokens)
print("Test perplexity: {:.02f}".format(2**(-1*log_p_data/num_real_tokens)))

Train perplexity: 21.28


<ipython-input-7-5e1b63fcb65f>:51: RuntimeWarning: divide by zero encountered in log2
  s = np.log2(self.next_word_proba(seq[i], seq[i-2:i]))


Test perplexity: inf


What's going on here? Our model gets an absurdly low perplexity on the training data, but a perplexity of _infinity_ on the test data.

**Answer:** the n-gram model badly overfits without any smoothing.

## Smoothing and Handling the Unknown

Our simple model doesn't have any real mechanism for handling unknown words - if we feed something unseen to `score_seq`, it will assign it a probability of zero:

In [16]:
lm.score_seq(["<s>", "i", "love", "w266", "</s>"])[0]

<ipython-input-7-5e1b63fcb65f>:51: RuntimeWarning: divide by zero encountered in log2
  s = np.log2(self.next_word_proba(seq[i], seq[i-2:i]))


-inf

This is the cause of the infinite perplexity, above: $\log_2 0 = -\infty $. 

**Exercise:** besides unknown words, when else would the un-smoothed trigram model predict $p(w\ |\ w_{i-1}, w_{i-2}) = 0$ ?

Is assuming zero probabilities realistic? Let's look back at our unigram distribution:

In [17]:
print("% words seen only once: {:.02%}".format(sum(counts * (counts == 1)) / sum(counts)))

NameError: name 'counts' is not defined

About 1 in 50 words were seen only once in our corpus, so we might expect a comparable fraction of words in a new sample to also be previously-unseen:

In [ ]:
print("% <unk> in test set: {:.02%}".format(np.sum(np.array(test_tokens) == "<unk>") / len(test_tokens)))

If we want to use our language model in the wild, we'll need to implement some kind of smoothing to hedge our bets whenever these come up. This will be a major focus of Assignment 3, in which you'll build on the `SimpleTrigramLM` by implementing Laplace (add-k) and Kneser-Ney smoothing.